# KNN final!

In [ ]:
%%capture --no-display

from skforecast.model_selection_multiseries import grid_search_forecaster_multivariate
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# print(ventas2['FechaReporte'].max())

# product = '100003932'  # Replace with the desired product code
print("len products",len(ventas2['productCode'].unique()))
# datos_series=[]
productos=ventas2['productCode'].unique()
predicciones_fut=[]      
# models=[]      # mse train: models[i]['mean_squared_error']

error_codes_fut=[]

# param_grid={'n_estimators':[50,80,100],'max_depth':[2,5,10],'min_samples_leaf':[5,10]}
param_grid={'n_neighbors':[2,3,4,5,6,7,8]}     #param grid for cv - gridsearch

for i,product in enumerate(ventas2['productCode'].unique()):
    # print(f"product{i}: {product}")
    # if i>2:
    #     break
    product_data = ventas2[ventas2['productCode'] == product]
    product_data=product_data.set_index('FechaLunesSemana')
    # product_data=product_data.asfreq('W')
    product_data=product_data.sort_index()

    # datos_series.append(product_data)   # append to series list by product for visualization

    steps=12    # weekly ->3months is 12 steps
    datos_train=product_data[:-steps]
    datos_test=product_data[-steps:]
    # print(datos_train.head())
    # print(datos_train.shape)

    try:
        forecaster2=ForecasterAutoregMultiVariate(
                        regressor=KNeighborsRegressor(),
                        level='SumaCantidadSemanal',
                        steps=steps,
                        lags=1
                    )


        results_grid=grid_search_forecaster_multivariate(
                        forecaster=forecaster2,
                        series=datos_train[['PromedioDiasSinVenta','SumaCantidadSemanal','puti','SumaUtilidadSemanal']],
                        param_grid=param_grid,
                        steps=steps,
                        refit=True,
                        metric='mean_squared_error',
                        initial_train_size=len(datos_train[:-steps*2]),
                        fixed_train_size=False,
                        return_best=True,
                        verbose=False
                    )

        # results_grid.reset_index(drop=True,inplace=True)
        # models.append(results_grid.loc[0])

        predictions_future=forecaster2.predict_interval(last_window=datos_test[['PromedioDiasSinVenta','SumaCantidadSemanal','puti','SumaUtilidadSemanal']])
        predictions_future['Codigo']=product
        predictions_future.reset_index(names=['Dates'],inplace=True)
        predictions_future['Dates']=datos_train.index.to_list()[-1]+pd.to_timedelta(predictions_future['Dates'],unit='W')
        # print(predictions_future)
        predicciones_fut.append(predictions_future)

    except:
        error_codes_fut.append(product)

In [ ]:
len(predicciones_fut)

4481

In [ ]:
import json
data = {}

for i in range(len(predicciones_fut)):
    codigo = predicciones_fut[i]['Codigo'][0]    #predicciones [df[pred,pred5,pred95,fecha,Codigo],df,df,....]

    nombre = code_names.loc[code_names['productCode']==codigo,'name'].iloc[0]  #get name from code

    if nombre not in data:
        data[nombre] = {'mse': "",
                        'n_neighbors':"",
                        'time_series_dates': [],
                        'time_series_values':[],
                        'prediction_dates': [],
                        'prediction_values':[],
                        'confianza5': [],
                        'confianza95':[],
                        'codigo': "",
                        'nombre': ""}
            
    data[nombre]['time_series_values'].extend(datos_series[i]['SumaCantidadSemanal'].values.tolist())
    fechas = datos_series[i].index.to_list()
    fechas_str = [fecha.strftime('%Y-%m-%d %H:%M:%S') for fecha in fechas]
    data[nombre]['time_series_dates'].extend(fechas_str)

    data[nombre]['prediction_dates'].extend([d.split('T')[0] for d in predicciones_fut[i]['Dates'].values.astype(str)])
    data[nombre]['prediction_values'].extend(predicciones_fut[i]['SumaCantidadSemanal'].values.tolist())
    data[nombre]['confianza5'].extend(predicciones_fut[i]['lower_bound'].values.tolist())
    data[nombre]['confianza95'].extend(predicciones_fut[i]['upper_bound'].values.tolist())

    data[nombre]['codigo']=codigo
    data[nombre]['nombre']=nombre

# Guardar el diccionario en un archivo JSON
with open('jc_model_results_future.json', 'w') as f:
    json.dump(data, f)

In [ ]:
total_mse=[]
for item in data:
    total_mse.append(float(data[item]['mse']))

sum(total_mse)/len(total_mse)

46.27535846086328

In [ ]:
float(data['2 ACID MANTLE N LOCION 120 ML P.E']['mse'])

0.05272108843537415

In [ ]:
print(len(data['2 ACID MANTLE N LOCION 120 ML P.E']['confianza5']))
print(len(data['2 ACID MANTLE N LOCION 120 ML P.E']['confianza95']))
print(len(data['2 ACID MANTLE N LOCION 120 ML P.E']['prediction_dates']))
print(len(data['2 ACID MANTLE N LOCION 120 ML P.E']['prediction_values']))
print(len(data['2 ACID MANTLE N LOCION 120 ML P.E']['time_series_dates']))
print(len(data['2 ACID MANTLE N LOCION 120 ML P.E']['time_series_values']))

12
12
12
12
57
57


In [ ]:
key='LORATADINA 10 MG 20 TABLETAS AG'

In [ ]:
series_data=pd.DataFrame(index=data[key]['time_series_dates'],data={'serie':data[key]['time_series_values']})

In [ ]:
predict_data=pd.DataFrame(index=data[key]['prediction_dates'],
                          data={'pred':data[key]['prediction_values'],
                                'conf5':data[key]['confianza5'],
                                'conf95':data[key]['confianza95']})

In [ ]:
with open('jc_model_results_future.json', 'w') as f:
    json.dump(data, f)

In [ ]:
len(predicciones_fut)

4481

In [ ]:
series_data.head()

,serie
2022-03-19 00:00:00,0
2022-03-26 00:00:00,1
2022-04-02 00:00:00,0
2022-04-02 00:00:00,0
2022-04-09 00:00:00,0


In [ ]:
data_testing = {}

for i in range(1):
    codigo = predicciones_fut[i]['Codigo'][0]    #predicciones [df[pred,pred5,pred95,fecha,Codigo],df,df,....]

    nombre = code_names.loc[code_names['productCode']==codigo,'name'].iloc[0]  #get name from code

    if nombre not in data_testing:
        data_testing[nombre] = {'mse': "",
                        'n_neighbors':"",
                        'time_series_dates': [],
                        'time_series_values':[],
                        'prediction_dates': [],
                        'prediction_values':[],
                        'confianza5': [],
                        'confianza95':[],
                        'codigo': "",
                        'nombre': ""}
            
    data_testing[nombre]['time_series_values'].extend(datos_series[i]['SumaCantidadSemanal'].values.tolist())
    fechas = datos_series[i].index.to_list()
    fechas_str = [fecha.strftime('%Y-%m-%d %H:%M:%S') for fecha in fechas]
    data_testing[nombre]['time_series_dates'].extend(fechas_str)

    data_testing[nombre]['prediction_dates'].extend([d.split('T')[0] for d in predicciones_fut[i]['Dates'].values.astype(str)])
    data_testing[nombre]['prediction_values'].extend(predicciones_fut[i]['SumaCantidadSemanal'].values.tolist())
    data_testing[nombre]['confianza5'].extend(predicciones_fut[i]['lower_bound'].values.tolist())
    data_testing[nombre]['confianza95'].extend(predicciones_fut[i]['upper_bound'].values.tolist())

    data_testing[nombre]['codigo']=codigo
    data_testing[nombre]['nombre']=nombre